<a href="https://colab.research.google.com/github/CarlTeapot/Walmart-Recruiting/blob/main/model_experiment_lightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mlflow==2.2.2
!pip install dagshub

In [ ]:
!pip install setuptools==65.5.0


In [ ]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/ML PROEQTI/clean_train_data.csv')

In [ ]:
train_df

,Store,Dept,Date,Weekly_Sales,IsHoliday,Type,Size,Temperature,Fuel_Price,MarkDown1,...,MarkDown4,MarkDown5,CPI,Unemployment,Season,Month,Year,Week,Holiday_in_1_week_lag,Weeks_until_next_holiday
0,1,1,2010-02-05,24924.50,0,A,151315,42.31,2.572,NaN,...,NaN,NaN,211.096358,8.106,Winter,2,2010,5,1.0,1
1,1,1,2010-02-12,46039.49,1,A,151315,38.51,2.548,NaN,...,NaN,NaN,211.242170,8.106,Winter,2,2010,6,0.0,30
2,1,1,2010-02-19,41595.55,0,A,151315,39.93,2.514,NaN,...,NaN,NaN,211.289143,8.106,Winter,2,2010,7,0.0,29
3,1,1,2010-02-26,19403.54,0,A,151315,46.63,2.561,NaN,...,NaN,NaN,211.319643,8.106,Winter,2,2010,8,0.0,28
4,1,1,2010-03-05,21827.90,0,A,151315,46.50,2.625,NaN,...,NaN,NaN,211.350143,8.106,Spring,3,2010,9,0.0,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420207,45,98,2012-09-28,508.37,0,B,118221,64.88,3.997,4556.61,...,1601.01,3288.25,192.013558,8.684,Fall,9,2012,39,0.0,0
420208,45,98,2012-10-05,628.10,0,B,118221,64.89,3.985,5046.74,...,2253.43,2340.01,192.170412,8.667,Fall,10,2012,40,0.0,0
420209,45,98,2012-10-12,1061.02,0,B,118221,54.47,4.000,1956.28,...,599.32,3990.54,192.327265,8.667,Fall,10,2012,41,0.0,0
420210,45,98,2012-10-19,760.01,0,B,118221,56.47,3.969,2004.02,...,437.73,1537.49,192.330854,8.667,Fall,10,2012,42,0.0,0


In [ ]:
train_df.dtypes

,0
Store,int64
Dept,int64
Date,object
Weekly_Sales,float64
IsHoliday,int64
Type,object
Size,int64
Temperature,float64
Fuel_Price,float64
MarkDown1,float64


In [ ]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import mlflow
import mlflow.sklearn



df = train_df.copy()
df['Type'] = df['Type'].astype('category')
df['Season'] = df['Season'].astype('category')
X = df.drop(columns=['Weekly_Sales'])
y = df['Weekly_Sales']

split_index = int(len(X) * 0.8)

X_train = X[:split_index]
X_val = X[split_index:]

y_train = y[:split_index]
y_val = y[split_index:]

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import lightgbm as lgb

class DropColumns(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_drop):
        self.columns_to_drop = columns_to_drop

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.drop(columns=self.columns_to_drop)

preprocessing = Pipeline(steps=[
    ('drop_columns', DropColumns(columns_to_drop=[
        'Date',
    ])),
])

In [ ]:
model = lgb.LGBMRegressor(
    n_estimators=100,
    learning_rate=0.1,
    num_leaves=63,
    reg_lambda=1,
    reg_alpha=0.1,
    n_jobs=-1,
    objective='regression_l1',
)
preprocessing = Pipeline(steps=[
    ('drop_columns', DropColumns(columns_to_drop=[
        'Date',
    ])),
])



sample_weight = X_train['IsHoliday'].astype(int).replace(1, 5).replace(0, 1)

pipeline = Pipeline(steps=[
    ('preprocessing', preprocessing),
    ('model', model)
])


pipeline.fit(X_train, y_train, model__sample_weight=sample_weight)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.019177 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2548
[LightGBM] [Info] Number of data points in the train set: 336169, number of used features: 20
[LightGBM] [Info] Start training from score 8295.540039


,steps,"[('preprocessing', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,steps,"[('drop_columns', ...)]"
,transform_input,None
,memory,None
,verbose,False
,columns_to_drop,['Date']
,boosting_type,'gbdt'
,num_leaves,63


In [ ]:
X_train

,Store,Dept,Date,IsHoliday,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Weeks_until_next_holiday,Season
138466,15,3,2011-04-01,0,B,123737,30.34,3.811,NaN,NaN,NaN,NaN,NaN,134.068258,7.658,23,1
289214,30,25,2010-06-11,0,C,42988,83.51,2.668,NaN,NaN,NaN,NaN,NaN,211.112002,8.200,13,2
52351,6,27,2010-06-04,0,A,202505,79.44,2.705,NaN,NaN,NaN,NaN,NaN,212.698244,7.092,14,2
203504,21,49,2011-12-02,0,B,140167,48.72,3.172,3389.10,43.0,1258.32,325.35,8623.67,218.359032,7.441,4,0
233606,24,55,2012-01-06,0,A,203819,32.86,3.585,7325.68,25367.9,203.51,1745.20,3261.35,136.698129,8.659,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259178,27,18,2012-10-19,0,A,204184,56.53,4.153,2639.32,NaN,4.77,575.48,3147.23,142.863363,8.000,0,3
365838,39,36,2010-05-21,0,A,184109,76.67,2.826,NaN,NaN,NaN,NaN,NaN,209.392294,8.464,16,1
131932,14,29,2010-04-30,0,A,200898,53.15,2.921,NaN,NaN,NaN,NaN,NaN,181.662036,8.899,19,1
146867,15,90,2011-07-01,0,B,123737,67.43,3.916,NaN,NaN,NaN,NaN,NaN,135.446800,7.806,10,2


In [ ]:
from sklearn.metrics import mean_absolute_error

def wmae(y_true, y_pred, weights):
    return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)

# Predictions
y_train_pred = pipeline.predict(X_train)
y_val_pred = pipeline.predict(X_val)

# Weights
train_weights = X_train['IsHoliday'].replace(1, 5).replace(0, 1)
val_weights = X_val['IsHoliday'].replace(1, 5).replace(0, 1)

# WMAE
wmae_train = wmae(y_train, y_train_pred, train_weights)
wmae_val = wmae(y_val, y_val_pred, val_weights)

print(f"Train WMAE: {wmae_train}")
print(f"Validation WMAE: {wmae_val}")

/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:1085: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  with _raise_or_warn_if_not_fitted(self):
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:1085: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  with _raise_or_warn_if_not_fitted(self):


Train WMAE: 3800.3154786841214
Validation WMAE: 4542.555695923023


In [ ]:

name = "XGBoost with train test split and lag features"


with mlflow.start_run(run_name = name):
        mlflow.sklearn.log_model(pipeline, "best_model")
        mlflow.log_metric("Validation WMAE ", wmae_val)
        mlflow.log_metric("Train WMAE", wmae_train)


2025/08/01 22:19:01 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2023-02-28; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'rapids-dask-dependency'}


In [ ]:
import mlflow
import dagshub

dagshub.init(repo_owner='CarlTeapot', repo_name='Walmart-Recruiting', mlflow=True)

Accessing as CarlTeapot

Initialized MLflow to track repo "CarlTeapot/Walmart-Recruiting"

Repository CarlTeapot/Walmart-Recruiting initialized!

In [ ]:
import itertools
import numpy as np
import mlflow
import mlflow.sklearn
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

def weighted_mean_absolute_error(y_true, y_pred, weights):
    return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)


param_grid = {
    'model__n_estimators': [100, 500],
    'model__num_leaves': [31, 63],
    'model__learning_rate': [0.05, 0.1],
    'model__reg_alpha': [0.1, 1],
    'model__reg_lambda': [0.1, 1]
}


run_count = 0




for params in list(itertools.product(*param_grid.values())):
    run_count += 1
    current_params = dict(zip(param_grid.keys(), params))

    model = lgb.LGBMRegressor(
        objective='regression_l1',
        random_state=42,
        **{k.replace('model__', ''): v for k, v in current_params.items()}
    )

    pipeline = Pipeline(steps=[
        ('preprocessing', preprocessing),
        ('model', model)
    ])

    print(f"Starting run {run_count} with parameters: {current_params}")

    with mlflow.start_run(run_name=f"Walmart Sales forecasting LightGBM run {run_count}"):

        mlflow.log_params(current_params)


        sample_weight_train = X_train['IsHoliday'].astype(int).replace(1, 5).replace(0, 1)
        pipeline.fit(X_train, y_train, model__sample_weight=sample_weight_train)


        y_train_pred = pipeline.predict(X_train)
        y_val_pred = pipeline.predict(X_val)

        sample_weight_val = X_val['IsHoliday'].astype(int).replace(1, 5).replace(0, 1)


        train_wmae = weighted_mean_absolute_error(y_train, y_train_pred, sample_weight_train)
        val_wmae = weighted_mean_absolute_error(y_val, y_val_pred, sample_weight_val)

        print(f"  Train WMAE: {train_wmae}")
        print(f"  Validation WMAE: {val_wmae}")

        mlflow.log_metric("Train WMAE", train_wmae)
        mlflow.log_metric("Validation WMAE", val_wmae)

        mlflow.sklearn.log_model(pipeline, f"pipeline_run_{run_count}")

    print("-" * 50)

print("\nHyperparameter tuning complete. Check DagsHub for results.")

Starting run 1 with parameters: {'model__n_estimators': 100, 'model__num_leaves': 31, 'model__learning_rate': 0.05, 'model__reg_alpha': 0.1, 'model__reg_lambda': 0.1}
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.035754 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2549
[LightGBM] [Info] Number of data points in the train set: 336169, number of used features: 20
[LightGBM] [Info] Start training from score 8295.540039


/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:1085: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  with _raise_or_warn_if_not_fitted(self):
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:1085: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  with _raise_or_warn_if_not_fitted(self):


  Train WMAE: 5648.179472913347
  Validation WMAE: 5242.481531298186


2025/08/01 22:20:15 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2023-02-28; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'rapids-dask-dependency'}


--------------------------------------------------
Starting run 2 with parameters: {'model__n_estimators': 100, 'model__num_leaves': 31, 'model__learning_rate': 0.05, 'model__reg_alpha': 0.1, 'model__reg_lambda': 1}
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.036213 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2549
[LightGBM] [Info] Number of data points in the train set: 336169, number of used features: 20
[LightGBM] [Info] Start training from score 8295.540039


...

--------------------------------------------------
Starting run 32 with parameters: {'model__n_estimators': 500, 'model__num_leaves': 63, 'model__learning_rate': 0.1, 'model__reg_alpha': 1, 'model__reg_lambda': 1}
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.024165 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2549
[LightGBM] [Info] Number of data points in the train set: 336169, number of used features: 20
[LightGBM] [Info] Start training from score 8295.540039


/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:1085: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  with _raise_or_warn_if_not_fitted(self):
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:1085: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  with _raise_or_warn_if_not_fitted(self):


  Train WMAE: 2740.781119538149
  Validation WMAE: 4384.8427133704245


2025/08/01 22:46:47 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2023-02-28; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'rapids-dask-dependency'}


--------------------------------------------------

Hyperparameter tuning complete. Check DagsHub for results.
